In [53]:
from tqdm.auto import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer, GPT2LMHeadModel

In [32]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")

model = GPT2LMHeadModel.from_pretrained("/users/ujan/caricatures/models/scan/distilgpt2_19k")

model.generation_config.max_length = 256

In [33]:
dataset = load_dataset("scan", "simple", trust_remote_code=True)

In [ ]:
dataset['test'][4]

In [41]:
inputs = tokenizer(
    "run thrice and walk opposite left<sep>", return_tensors="pt")
tokenizer.decode(model.generate(**inputs)[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'run thrice and walk opposite left <sep>  I_RUN I_RUN I_RUN I_TURN_LEFT I_TURN_LEFT I_WALK <|endoftext|>'

In [ ]:
test_dataset = dataset['test'].select(range(100))

In [54]:
count = 0
bar = tqdm(range(len(test_dataset)))
for example in test_dataset:
    command = example['commands']
    label = example['actions']
    inputs = tokenizer(command+"<sep>", return_tensors="pt")
    output = model.generate(**inputs)[0]
    #output = model.generate(**inputs, num_beams=5, max_new_tokens=256)[0]
    output = tokenizer.decode(output, skip_special_tokens=True).split("<sep>")[-1].strip()
    if output == label:
        count += 1
    bar.update(1)

print(count/len(test_dataset))

  0%|          | 0/4182 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


KeyboardInterrupt: 

In [ ]:
# distilgpt2 : acc = 0.6406025824964132
# distilgpt2 : acc = 0.67 -> beam search (beam size = 3)
# gpt2 : acc = 0.810856049736968